In [17]:
import __init__

In [18]:
#%load_ext autoreload
#%autoreload 2

In [19]:
# Load external librairies
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style("whitegrid")

# Load internal librairies
from fleming_lib.dataset import create_dataset
from fleming_lib.severity_scores import compute_sapsii_score
from fleming_lib.utils import add_categories

from fleming_lib.metrics import add_target, add_age

In [20]:
# Open a connection to OMOP
from fleming_lib.tools import connect_to_omop
conn = connect_to_omop()

In [21]:
import time
import warnings

from fleming_lib.metrics import add_rolling_avg, add_target, add_super_target, add_age
from fleming_lib.utils import to_categorical, to_onehot, to_numeric, convert_frac, add_categories, add_missing_columns, check_length

def create_dataset(list_patients, n_patients_per_batch=10, verbose=False):
    """Create list of dataset given a list of patients.
    
    Parameters
    ----------
    list_patients : list of int
        List of patients ID.
    n_patients_per_batch : int (default=10)
        Number of patients to sequentially load data for, in order not to cause timeout if the query is too long to
        process by the server.
    verbose : bool (default=False)
        Verbosity level.
    
    Returns
    -------
    dataset : pd.DataFrame
        Dataset containing all data associated to each patient.
        
    """
    t0 = time.time()
    frame = []
    
    if not isinstance(list_patients, list):
        list_patients = [list_patients]
        
    n_patients = len(list_patients)
    
    # Extract meta data
    # -----------------
    if verbose:
        msg = 'Extracting meta data...'
        delta_t = str(int(time.time() - t0)) + ' s'
        print('{:100s} [{:10s}]'.format(msg, delta_t), end='\r')
            
    query = """     
    
        select 
            distinct p.person_id, p.gender_source_value gender, p.race_source_value race, p.birth_datetime 
        from person p
        
        ;"""
    print('   start meta...        ', end='\r')
    meta = pd.read_sql_query(query, conn)
    print('...end meta            ', end='\r')
    # Dictionary containing unique categories for each categorical variable
    categories = dict()
    
    # Convert categorical variable to 'categorical' type
    categorical_variables = ['gender', 'race']
    # Extracting categories for each categorical variable
    categories = add_categories(categories, meta, categorical_variables)
    
    meta = to_categorical(meta, categorical_variables, categories)
    meta = to_onehot(meta, categorical_variables)
    # One-hot column names
    meta_names = meta.columns
    
    # Extract unique measurements values from categorical variables
    # (here 'Heart rate rhythm' and 'Admission Type')
    query = """
        select
            distinct m.measurement_concept_name, m.value_source_value
        from 
            measurement m 
        where
            m.measurement_concept_id in
            (3022318   -- heart_rhythm
            )
        order by
            m.measurement_concept_name, m.value_source_value     
        ;"""

    unique_categ_values = pd.read_sql_query(query, conn)
    categorical_variables = ['Heart rate rhythm']
    # Adding categories of each categorical variables to dict 'categories'
    for var in categorical_variables:
        tmp = unique_categ_values.loc[unique_categ_values.measurement_concept_name == var]
        tmp.drop('measurement_concept_name', axis=1, inplace=True)
        tmp.rename(index=str, columns={'value_source_value': var}, inplace=True)
        categories = add_categories(categories, tmp, var)
    
    # Create sublist of patients (batch)
    n_iter = int(n_patients / n_patients_per_batch)
    sublists_patients = [list_patients[i: i+n_patients_per_batch]
                         for i in range(0, n_patients, n_patients_per_batch)]
    n_sublists = len(sublists_patients)
    
    # Extracting data for each patient
    for i, sublist_patients in enumerate(sublists_patients):
        if verbose:
            base_msg = 'Batch {}/{}'.format(i+1, n_sublists)
            msg = base_msg
            delta_t = str(int(time.time() - t0)) + ' s'
            print('{:100s} [{:10s}]'.format(msg, delta_t), end='\r')
        
        # Extract measures
        # ----------------
        if verbose:
            add_msg = 'Extracting measures...'
            msg = base_msg + ' - ' + add_msg
            delta_t = str(int(time.time() - t0)) + ' s'
            print('{:100s} [{:10s}]'.format(msg, delta_t), end='\r')
            
        if len(sublist_patients) == 1:
            match_person = "m.person_id = {}".format(sublist_patients[0])
        else:
            match_person = "m.person_id in {}".format(tuple(sublist_patients))
            
        query = """ 
        with icustays as
        ( 

        select person_id
        from visit_detail
        where visit_detail_concept_id = 581382              
        and visit_type_concept_id = 2000000006 

        ) ,

        surgflag as
        (
          select icustays.person_id as person_id
            , case when c.concept_id  = 4149152 then 1 else 0 end as surgical
            , row_number() over
            (
              partition by icustays.person_id
              order by vd.visit_start_datetime
            ) as serviceOrder

          from icustays
          left join visit_detail vd
              on icustays.person_id = vd.person_id 

          left join concept c
              on c.concept_id = vd.visit_detail_concept_id 

          where vd.visit_type_concept_id = 45770670   -- 'Services and care'

        ) ,

        admissions_icu as (

        select 
            icustays.person_id as person_id,
            vd.visit_detail_id visit_detail_id,
            case
                  when c.concept_id  = 4149152 and v.visit_source_value = 'ELECTIVE' -- and sf.surgical = 1
                  -- c.concept_id is for 'Surgical service' and v.visit_source_value for 'Scheduled' ones
                      then 'ScheduledSurgical'

                  when c.concept_id  = 4149152 and (v.visit_source_value = 'EMERGENCY' or v.visit_source_value = 'URGENT') -- and sf.surgical = 1
                  -- c.concept_id is for 'Surgical service' and v.visit_source_value for 'Unscheduled' ones
                      then 'UnscheduledSurgical'

                  when c.concept_id  = 45763735 -- General medical service 
                      then 'Medical'

                  else 'No matching found' 
                  
            end as admission_type

        from icustays

        left join visit_detail vd
            on icustays.person_id = vd.person_id 

        left join concept c
            on c.concept_id = vd.visit_detail_concept_id   

        left join visit_occurrence v            -- WHEN ADDED,number_icu_general_medical = 112 370 ---> 697 338 !
            on icustays.person_id = v.person_id

        -- left join surgflag sf
        --    on icustays.person_id = sf.person_id and sf.serviceOrder = 1

        where vd.visit_type_concept_id = 45770670   -- 'Services and care', maybe redondant.. 

        )    
        
            
        select
            distinct m.person_id, m.measurement_datetime, m.measurement_concept_name, m.value_source_value, 
            m.unit_source_value, d.death_datetime, adm_icu.admission_type admission_type
        
        from 
            measurement m
        left join 
            admissions_icu adm_icu on m.visit_detail_id = adm_icu.visit_detail_id
        left join 
            death d on d.person_id = m.person_id
            
        where
            measurement_concept_id IN
            (3022318,   -- heart_rhythm
             3024171,   -- respiratory_rate
             3028354,   -- vent_settings
             3012888,   -- diastolic_bp
             3027598,   -- map_bp
             3004249,   -- systolic_bp
             3027018,   -- heart_rate
             3020891,   -- temperature
             3016502,   -- spo2
             3020716,   -- fio2
             3032652,   -- glasgow coma scale
             
             -- chemicals
             3019550,   -- sodium serum/plasma (en) | natremie (fr)
             3023103,   -- potassium serum/plasma (en) | kaliemie (fr)
             3024128,   -- bilirubin
             
             -- hemato
             3003282    -- Leukocytes [#/volume] in Blood by Manual count
             
            )
        and {}
        order by measurement_datetime
            ;""".format(match_person)
        
        df = pd.read_sql_query(query, conn)
        
        # Check if data is empty for a patient
        check_length(df)

        if verbose:
            add_msg = 'Formatting data...'
            msg = base_msg + ' - ' + add_msg
            delta_t = str(int(time.time() - t0)) + ' s'
            print('{:100s} [{:10s}]'.format(msg, delta_t), end='\r')

        df['death_datetime'] = pd.to_datetime(df['death_datetime'])
        df['measurement_datetime'] = pd.to_datetime(df['measurement_datetime'])
        
        # Add target: patient's death' status, relative to the measurement datetime (target) and to the hospital stay (super_target)
        df = df.groupby('person_id').apply(add_target)
        df = df.groupby('person_id').apply(add_super_target)
        
        # Convert to timeseries matrix
        df = df.pivot_table(index= ['measurement_datetime','target','super_target','person_id'],#'admission_type'], 
                            columns='measurement_concept_name', values='value_source_value', aggfunc='first')
        df.reset_index(inplace=True)
        df.columns.name = None
        
        # Convert types
        # -------------
        # Convert to numerical
        numerical_variables = ['BP diastolic', 'BP systolic', 'Body temperature', 'Heart rate', 'Mean blood pressure',
                               'Glasgow coma scale', 'Oxygen concentration breathed', 
                               'Mean pressure Respiratory system airway Calculated', 
                               'Oxygen saturation in Arterial blood', 'Respiratory rate',
                               'Leukocytes [#/volume] in Blood by Manual count',
                               'Potassium serum/plasma', 'Sodium serum/plasma', 'Total Bilirubin serum/plasma']
           
        df = add_missing_columns(df, numerical_variables)
    
        df = convert_frac(df, numerical_variables)
        df = to_numeric(df, numerical_variables)
        
        # Convert to categorical and one-hot encode
        categorical_variables = ['Heart rate rhythm', 'admission_type']
        
        df = add_missing_columns(df, categorical_variables)
        
        df = to_categorical(df, categorical_variables, categories)
        df = to_onehot(df, categorical_variables)
        
        
        # Add meta data to measures
        # -------------------------
        if verbose:
            add_msg = 'Adding meta data...'
            msg = base_msg + ' - ' + add_msg
            delta_t = str(int(time.time() - t0)) + ' s'
            print('{:100s} [{:10s}]'.format(msg, delta_t), end='\r')
            
        df = pd.merge(df, meta, how='inner', on='person_id')
        
        # Add additional features
        # -----------------------
        if verbose:
            add_msg = 'Adding additional features...'
            msg = base_msg + ' - ' + add_msg
            delta_t = str(int(time.time() - t0)) + ' s'
            print('{:100s} [{:10s}]'.format(msg, delta_t), end='\r')
        
        # - age
        df = df.groupby('person_id').apply(add_age, round_to_dec=1)
        # - 2h rolling average respiratory rate
        df = df.groupby('person_id').apply(add_rolling_avg, column='Respiratory rate', window=2)
        
        frame.append(df)
        
        if verbose:
            add_msg = 'Done'
            msg = base_msg + ' - ' + add_msg
            delta_t = str(int(time.time() - t0)) + ' s'
            print('{:100s} [{:10s}]'.format(msg, delta_t), end='\r')
            print('')
            
    # Concat dataframes
    dataset = pd.concat(frame)
    dataset = dataset.reindex_axis(frame[0].columns, axis=1)
    
    return dataset

In [22]:
# Define list of patients
single_id = 62063368
list_patients = [62063368, 62063384, 62063393]

# Create dataset
dataset = create_dataset(list_patients, verbose=True, n_patients_per_batch=10)

/Users/jeremydesir/Desktop/DataForGood/batch4_diafoirus_fleming/fleming_lib/utils.py:257: UserWarning: Colmun `admission_type` missing: adding it and filling with NaN.
  warnings.warn(wrn)


Batch 1/1 - Done                                                                                     [47 s      ]


/Users/jeremydesir/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:307: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.


In [15]:
df = dataset.copy(deep=True)

In [16]:
df.head()

,measurement_datetime,target,super_target,person_id,BP diastolic,BP systolic,Body temperature,Glasgow coma scale,Heart rate,Leukocytes [#/volume] in Blood by Manual count,...,race_UNABLE TO OBTAIN,race_UNKNOWN/NOT SPECIFIED,race_WHITE,race_WHITE - BRAZILIAN,race_WHITE - EASTERN EUROPEAN,race_WHITE - OTHER EUROPEAN,race_WHITE - RUSSIAN,race_NaN,age,Respiratory rate avg h-2
0,2182-07-31 04:11:00,0,1,62063393,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,1,0,0,0,0,0,60.8,NaN
1,2182-07-31 04:37:00,0,1,62063393,NaN,NaN,NaN,NaN,NaN,15.1,...,0,0,1,0,0,0,0,0,60.8,NaN
2,2182-07-31 04:45:00,0,1,62063393,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,1,0,0,0,0,0,60.8,NaN
3,2188-11-12 07:15:00,0,1,62063368,NaN,NaN,NaN,NaN,NaN,14.4,...,0,0,0,0,0,0,0,0,23.9,NaN
4,2188-11-12 08:04:00,0,1,62063368,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,23.9,NaN


# The following has been done WITHOUT the admission_type causing the "Insufficient space" issue above (but 4 variables have been added to numerical)

In [ ]:
tmp = dataset.copy(deep=True)

In [ ]:
tmp.head(10)

In [ ]:
tmp = dataset.copy(deep=True)
tmp.iloc[:10,65:75]

In [ ]:
dfBodyTempC = tmp['Body temperature'].copy()
dfBodyTempC = pd.DataFrame([(tempF-32.0)/1.8 if tempF>65 else tempF for tempF in dfBodyTempC.values], index = tmp.index)
tmp = tmp.assign(bodyTemperature_C = dfBodyTempC.values)

In [ ]:
# Set relative time scale
#start_date = tmp['measurement_datetime'][0]
#tmp['measurement_datetime'] -= start_date

In [ ]:
# Let's set the index with measurement_datetime 
#tmp.set_index('measurement_datetime', inplace = True)

In [ ]:
tmp.head()

In [ ]:
from datetime import timedelta
from fleming_lib.preprocessing import fill_last_upto

In [ ]:
# Fill missing values with last one (up to h time prior to measurement)
tmp = tmp.groupby('person_id', group_keys=False).apply(fill_last_upto, h=timedelta(hours=24))

In [ ]:
tmp.head()

In [ ]:
# Sort index
tmp.sort_index(inplace=True)

In [ ]:
tmp.head()

In [ ]:
len(tmp)

In [ ]:
len(tmp.dropna())

In [ ]:
start_date = tmp['measurement_datetime'][0]
tmp['measurement_datetime'] -= start_date
tmp.set_index('measurement_datetime', inplace = True)
tmpd = tmp.dropna()

# Compute score (without admission type)

In [ ]:
tmpd.head()

In [ ]:
tmpd.shape

In [ ]:
tempRelevantComponents = tmpd[['age', 'Glasgow coma scale', 'BP systolic', 'bodyTemperature_C',
                               'Oxygen saturation in Arterial blood', 'Oxygen concentration breathed',
                               'Leukocytes [#/volume] in Blood by Manual count',
                               'Potassium serum/plasma', 'Sodium serum/plasma', 'Total Bilirubin serum/plasma']]

In [ ]:
tempRelevantComponentsScore = tempRelevantComponents.apply(compute_sapsii_score, axis=1)

In [ ]:
tempRelevantComponentsScore.plot(figsize=(16,9))

In [ ]:
tempRelevantComponents.head()